In [4]:
import os
import re
from pathlib import Path
from shutil import move

def get_label_filename(img_filename):
    base = os.path.splitext(img_filename)[0]
    if base.startswith(('cars_', 'pano_')):
        return base + '.txt'
    elif re.match(r'^\d', base):
        return 'cars_' + base + '.txt'
    else:
        return base + '.txt'

img_exts = {'.jpg', '.jpeg', '.png'}

# 처리할 하위 세트: train, val
for subset in ['train', 'test', 'val']:
    img_dir = Path(f'dataset/images/{subset}')
    lbl_dir = Path(f'dataset/labels/{subset}')

    for img_file in os.listdir(img_dir):
        src_img_path = img_dir / img_file
        if src_img_path.suffix.lower() not in img_exts:
            continue  # skip non-image files

        new_img_name = get_label_filename(img_file).replace('.txt', src_img_path.suffix)
        new_lbl_name = get_label_filename(img_file)

        if img_file != new_img_name:
            print(f"[INFO] ({subset}) 이미지 이름 변경: {img_file} -> {new_img_name}")
            move(str(src_img_path), str(img_dir / new_img_name))

            old_lbl_name = os.path.splitext(img_file)[0] + '.txt'
            old_lbl_path = lbl_dir / old_lbl_name
            new_lbl_path = lbl_dir / new_lbl_name

            if old_lbl_path.exists():
                print(f"[INFO] ({subset}) 라벨 이름 변경: {old_lbl_name} -> {new_lbl_name}")
                move(str(old_lbl_path), str(new_lbl_path))
            else:
                print(f"[WARNING] ({subset}) 라벨 파일 없음: {old_lbl_name}")

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'dataset\\images\\train'

In [2]:
import os

base_path = 'dataset'
splits = ['train', 'val', 'test']

summary = {}

for split in splits:
    img_dir = os.path.join(base_path, 'images', split)
    label_dir = os.path.join(base_path, 'labels', split)
    
    images = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
    labels = sorted([f for f in os.listdir(label_dir) if f.endswith('.txt')])
    
    missing_labels = []
    empty_labels = []

    for img in images:
        label_name = os.path.splitext(img)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)
        
        if not os.path.exists(label_path):
            missing_labels.append(img)
        else:
            with open(label_path, 'r') as f:
                content = f.read().strip()
                if not content:
                    empty_labels.append(img)
    
    summary[split] = {
        'image_count': len(images),
        'label_count': len(labels),
        'missing_labels': len(missing_labels),
        'empty_labels': len(empty_labels),
        'missing_label_files': missing_labels,
        'empty_label_files': empty_labels,
    }

for split, stats in summary.items():
    print(f"\n[{split.upper()} SET]")
    print(f"- 총 이미지 수: {stats['image_count']}")
    print(f"- 라벨 수: {stats['label_count']}")
    print(f"- 라벨 누락 이미지 수: {stats['missing_labels']}")
    print(f"- 빈 라벨 이미지 수: {stats['empty_labels']}")



[TRAIN SET]
- 총 이미지 수: 11584
- 라벨 수: 11584
- 라벨 누락 이미지 수: 0
- 빈 라벨 이미지 수: 3564

[VAL SET]
- 총 이미지 수: 1654
- 라벨 수: 1654
- 라벨 누락 이미지 수: 0
- 빈 라벨 이미지 수: 500

[TEST SET]
- 총 이미지 수: 3311
- 라벨 수: 3311
- 라벨 누락 이미지 수: 0
- 빈 라벨 이미지 수: 1003


In [3]:
import os

base_path = 'dataset'
splits = ['train', 'val', 'test']
deleted_files = []
skipped_files = []

for split in splits:
    img_dir = os.path.join(base_path, 'images', split)
    label_dir = os.path.join(base_path, 'labels', split)

    for label_file in os.listdir(label_dir):
        if not label_file.endswith('.txt'):
            continue

        label_path = os.path.join(label_dir, label_file)
        try:
            with open(label_path, 'r') as f:
                content = f.read().strip()
            if not content:  # 빈 라벨
                basename = os.path.splitext(label_file)[0]
                deleted_img = False
                for ext in ['.jpg', '.png', '.jpeg']:
                    img_path = os.path.join(img_dir, basename + ext)
                    if os.path.exists(img_path):
                        try:
                            os.remove(img_path)
                            deleted_files.append(img_path)
                            deleted_img = True
                        except Exception as e:
                            skipped_files.append((img_path, str(e)))
                        break

                try:
                    os.remove(label_path)
                    deleted_files.append(label_path)
                except Exception as e:
                    skipped_files.append((label_path, str(e)))
        except Exception as e:
            skipped_files.append((label_path, str(e)))

print(f"\n✅ 빈 라벨 이미지 및 라벨 총 {len(deleted_files)//2}개 세트 삭제 완료")
if skipped_files:
    print(f"⚠️ 삭제에 실패한 파일 {len(skipped_files)}개:")
    for f, reason in skipped_files:
        print(f"  - {f}: {reason}")



✅ 빈 라벨 이미지 및 라벨 총 5067개 세트 삭제 완료


In [8]:
import subprocess

command = [
    'python', 'train.py',
    '--img', '1280',
    '--batch', '16',
    '--epochs', '1000',
    '--data', 'data/car.yaml',
    '--cfg', 'models/yolov5s-amsffe.yaml',
    '--weights', '',
    '--hyp', 'data/hyps/hyp.amsffe.yaml',
    '--name', 'yolov5s_amsffe_lr0.005_epoch1000_img1280',
    '--project', 'runs/train',
    '--device', '0'
]

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

try:
    for line in process.stdout:
        print(line, end='')
except KeyboardInterrupt:
    print("\n⛔️ 학습 중단됨. Ctrl+C")

train: weights=, cfg=models/yolov5s-amsffe.yaml, data=data/car.yaml, hyp=data/hyps/hyp.amsffe.yaml, epochs=1000, batch_size=16, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_amsffe_lr0.005_epoch1000_img1280, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5  2025-7-9 Python-3.11.13 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

hyperparameters: lr0=0.005, lrf=0.2, momentum=0.95, weight_decay

In [2]:
import subprocess

command = [
    'python', 'val.py',
    '--data', 'data/car.yaml',                         # data.yaml에 test 경로 포함되어 있어야 함
    '--weights', 'runs/train/yolov5s_amsffe_lr0.001_epoch1000_img640/weights/best.pt',  # 평가할 모델
    '--img', '640',
    '--task', 'test',                                  # test 세트 평가
    '--batch-size', '4',
    '--device', '0',
    '--conf-thres', '0.1',
    '--name', 'yolov5s_amsffe_0.001_1000_test_img640_conf0.1_batch4',
    '--project', 'runs/test'
]

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

try:
    for line in process.stdout:
        print(line, end='')
except KeyboardInterrupt:
    print("\n⛔️ 중단됨.")

Traceback (most recent call last):
  File "c:\Users\MOON\Desktop\Project\ECM-Video\yolov5\val.py", line 31, in <module>
    from tqdm import tqdm
ModuleNotFoundError: No module named 'tqdm'


In [ ]:
import subprocess

detect_command = [
    'python', 'detect.py',
    '--weights', 'runs/train/yolov5s_amsffe_lr0.001_epoch1000_img640/weights/best.pt',
    '--source', 'dataset/images/inference',  # 이미지 폴더 경로
    '--img', '640',
    '--conf-thres', '0.1',
    '--save-txt',
    '--save-conf',
    '--project', 'runs/predict',
    '--name', 'amsffe_test'
]

subprocess.run(detect_command)


CompletedProcess(args=['python', 'detect.py', '--weights', 'runs/train/yolov5s_amsffe_lr0.005_epoch1000_img6402/weights/best.pt', '--source', 'dataset/images/inference', '--img', '640', '--conf-thres', '0.1', '--save-txt', '--save-conf', '--project', 'runs/predict', '--name', 'amsffe_test'], returncode=1)

In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp311-cp311-win_amd64.whl (2817.2 MB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp311-cp311-win_amd64.whl (5.5 MB)
Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp311-cp311-win_amd64.whl (4.1 MB)

   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ------------

In [1]:
!pip install -r requirements.txt

  Using cached matplotlib-3.10.3-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached scipy-1.16.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached ultralytics-8.3.163-py3-none-any.whl.metadata (37 kB)
  Using cached pandas-2.3.1-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached contourpy-1.3.2-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.5-cp311-cp311-win_amd64.whl.metadata (109 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached charset_normalizer-3.4.2-cp311-cp311-win_amd64.whl.metadata (36 kB)
  Using c

In [2]:
import torch
print(torch.cuda.is_available())  # True여야 함
print(torch.cuda.device_count())  # 최소 1 이상

False
0
